In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install faiss-cpu sentence-transformers transformers accelerate pdfplumber
!pip install rouge-score bert-score
!pip install Rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/5

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=bc8b4372e6454d0e3ba9dfadbb5f5b3353eff1e91b479770b69b363c286bcd17
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
import pdfplumber
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import AutoTokenizer, AutoModelForCausalLM
import faiss
import numpy as np
import torch
import os

# RAG

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# --- Step 1: Read NCERT Text from PDF ---
def load_pdf_text_from_directory(directory_path):
    full_text = ""
    # Loop through all files in the directory
    for filename in sorted(os.listdir(directory_path)):
        if filename.lower().endswith(".pdf"):
            file_path = os.path.join(directory_path, filename)
            print(f"Reading: {file_path}")

            # Extract text from the current PDF
            with pdfplumber.open(file_path) as pdf:
                for page in pdf.pages:
                    text = page.extract_text()
                    if text:
                        full_text += text + "\n"
    return full_text

# --- Step 2: Chunking ---
def chunk_by_sentences(text, chunk_size=100):
    sentences = sent_tokenize(text)
    chunks, current, wc = [], [], 0

    for sent in sentences:
        words = sent.split()
        if wc + len(words) > chunk_size:
            chunks.append(" ".join(current))
            current = []
            wc = 0
        current.append(sent)
        wc += len(words)

    if current:
        chunks.append(" ".join(current))

    return chunks

raw_text = load_pdf_text_from_directory("/content/drive/MyDrive/Dataset/Chemistry_Textbook/")
chunks = chunk_by_sentences(raw_text)
len(chunks)

Reading: /content/drive/MyDrive/Dataset/Chemistry_Textbook/lech101.pdf
Reading: /content/drive/MyDrive/Dataset/Chemistry_Textbook/lech102.pdf
Reading: /content/drive/MyDrive/Dataset/Chemistry_Textbook/lech103.pdf
Reading: /content/drive/MyDrive/Dataset/Chemistry_Textbook/lech104.pdf
Reading: /content/drive/MyDrive/Dataset/Chemistry_Textbook/lech105.pdf
Reading: /content/drive/MyDrive/Dataset/Chemistry_Textbook/lech201.pdf
Reading: /content/drive/MyDrive/Dataset/Chemistry_Textbook/lech202.pdf
Reading: /content/drive/MyDrive/Dataset/Chemistry_Textbook/lech203.pdf
Reading: /content/drive/MyDrive/Dataset/Chemistry_Textbook/lech204.pdf
Reading: /content/drive/MyDrive/Dataset/Chemistry_Textbook/lech205.pdf


1098

In [ ]:
# --- Step 3: Embed Chunks ---
embedder = SentenceTransformer("BAAI/bge-base-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# --- Step 4: Create FAISS Index ---
chunk_embeddings = embedder.encode(chunks, normalize_embeddings=True)
dim = chunk_embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(chunk_embeddings)
faiss.write_index(index, "/content/drive/MyDrive/Dataset/vector_store_faiss/chunk_index.faiss")

In [ ]:
# Load the index
index = faiss.read_index("/content/drive/MyDrive/Dataset/vector_store_faiss/chunk_index.faiss")

# Test-Train Data split

In [ ]:
import pandas as pd
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from sklearn.model_selection import train_test_split
import os
def load_all_csvs_from_folder(folder_path):
    all_dfs = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            print(f"Reading: {file_path}")
            df = pd.read_csv(file_path)
            all_dfs.append(df)

    # Combine all into one DataFrame
    combined_df = pd.concat(all_dfs, ignore_index=True)
    return combined_df

reference_df = load_all_csvs_from_folder("/content/drive/MyDrive/Dataset/Chemistry_Solutions_CSV_Subtopic_Clean/")

train_df, test_df = train_test_split(reference_df, test_size=0.2, random_state=42)

Reading: /content/drive/MyDrive/Dataset/Chemistry_Solutions_CSV_Subtopic_Clean/chapter_4_d_f_block_final.csv
Reading: /content/drive/MyDrive/Dataset/Chemistry_Solutions_CSV_Subtopic_Clean/chapter_5_coordination_compounds_final.csv
Reading: /content/drive/MyDrive/Dataset/Chemistry_Solutions_CSV_Subtopic_Clean/chapter_1_solutions_with_subtopics.csv
Reading: /content/drive/MyDrive/Dataset/Chemistry_Solutions_CSV_Subtopic_Clean/chapter_2_electrochemistry_with_subtopics.csv
Reading: /content/drive/MyDrive/Dataset/Chemistry_Solutions_CSV_Subtopic_Clean/chapter_3_chemical_kinetics_final.csv


In [ ]:
test_df.head()

,Sub Topic,Question,Answer
201,Temperature Dependence of the Rate of a Reaction,The rate constant for a first order reaction i...,"It is known that, Hence, the required..."
213,Collision Theory of Chemical Reactions,The decomposition of A into product has value ...,"From Arrhenius equation, we obtain Also, k1..."
138,Abnormal Molar Masses,100 g of liquid A (molar mass 140 g mol-1) was...,"Number of moles of liquid A, = 0.714 mol N..."
177,Rate of a Chemical Reaction,"For the reaction R → P, the concentration of a...",Average rate of reaction = 6.67 × 10 - 6M...
15,Electronic Configurations of the d-Block Elements,Name the oxometal anions of the first series o...,"(i) Vanadate, Oxidation state of V is + 5. ..."


In [ ]:
reference_questions = {}
for sub_topic in test_df['Sub Topic'].unique():
    ref_qs = reference_df[reference_df['Sub Topic'] == sub_topic]['Question'].tolist()
    reference_questions[sub_topic] = ref_qs

# Evaluation metrics

In [ ]:
import torch
from bert_score import score as bert_score
from rouge import Rouge
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

nltk.download('punkt')
nltk.download('punkt_tab')

# For each generated question, find best match score from references
def compute_bert_scores(gen, refs):
    P_all, R_all, F1_all = [], [], []
    for g in gen:
        P, R, F1 = bert_score([g]*len(refs), refs, lang="en", verbose=False)
        P_all.append(torch.max(P).item())
        R_all.append(torch.max(R).item())
        F1_all.append(torch.max(F1).item())
    return P_all, R_all, F1_all

rouge = Rouge()
def compute_rouge_scores(gen, refs):
    scores = []
    for g in gen:
        # Score against all references, take best
        best_score = max([rouge.get_scores(g, r)[0]["rouge-l"]["f"] for r in refs])
        scores.append(best_score)
    return scores

def compute_self_bleu(sentences, weights=(0.5, 0.5, 0, 0)):
    """
    Compute Self-BLEU score for a list of generated sentences.

    Args:
        sentences (List[str]): List of generated sentences.
        weights (Tuple): BLEU weights (e.g., (0.5, 0.5) for BLEU-2)

    Returns:
        float: Average Self-BLEU score (lower = more diverse)
    """
    scores = []
    smoothie = SmoothingFunction().method1

    for i in range(len(sentences)):
        hypothesis = nltk.word_tokenize(sentences[i])
        references = [nltk.word_tokenize(sent) for j, sent in enumerate(sentences) if j != i]

        if not references:
            continue

        score = sentence_bleu(references, hypothesis, weights=weights, smoothing_function=smoothie)
        scores.append(score)

    return sum(scores) / len(scores) if scores else 0.0

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Models

## Baseline Models

### RAG + QG LLM (T5)

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load model and tokenizer
# model_name = "valhalla/t5-base-qg-hl"
# Load tokenizer and model

def generate_t5_questions(model_name, context = "", num_questions = 5) -> list:
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    input_text = "generate question: " + "context:" + context.strip()
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)

    outputs = model.generate(
        input_ids=input_ids,
        #max_length=64,
        num_return_sequences=num_questions,
        do_sample=True,
        top_k=40,
        top_p=0.9,
        temperature=0.8,
        #early_stopping=True
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

In [ ]:
generated_questions = {}
for sub_topic in test_df['Sub Topic'].unique():
  # --- Step 5: RAG Query ---
  query = sub_topic
  query_embedding = embedder.encode([query], normalize_embeddings=True)

  top_k = 5
  scores, indices = index.search(query_embedding, top_k)
  retrieved_chunks = [chunks[i] for i in indices[0]]

  # --- Step 6: Rerank Results ---
  reranker = CrossEncoder("BAAI/bge-reranker-large")
  pairs = [[query, passage] for passage in retrieved_chunks]
  rerank_scores = reranker.predict(pairs)

  ranked_passages = [x for _, x in sorted(zip(rerank_scores, retrieved_chunks), reverse=True)]
  context = "\n".join(ranked_passages)

  # --- Step 7: Use context to generate question ---
  model_name = "iarfmoose/t5-base-question-generator"
  generated_qs = generate_t5_questions(model_name, context)
  for i, q in enumerate(generated_qs, 1):
    print(f"{i}. {q}")

  generated_questions[sub_topic] = generated_qs

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

1. how much heat does actinium generate?
2. What's the chemistry of the actinoids??
3. what type of activity is the chemistry of the actinoids?
4. The chemistry of the actinoids is more complicated than those of the lanthanoids
5. the chemistry of the actinoids is more complex than those of the lanthanoids
1. what does the structure of a coordination compound consist of?
2. Who was the first to formulate his ideas about the structures of coordination compounds?
3. Why did he formulate his theory of coordination compounds?
4. what type of coordination is the theory of?
5. what was his name?
1. which chemical composition is the most common?
2. what's the name of the periodic table?
3. the d–block occupies the large middle section of the periodic table PPPPPee
4. where is the d-block located?
5. what is the d-block??
1. how much pressure in the reaction is iodine?
2. True
3. True
4. True
5. How does the rate law calculate?
1. How do you determine the molar mass?
2. what is the dt?
3. what

In [ ]:
print(generated_questions)

{'The Actinoids': ['how much heat does actinium generate?', "What's the chemistry of the actinoids??", 'what type of activity is the chemistry of the actinoids?', 'The chemistry of the actinoids is more complicated than those of the lanthanoids', 'the chemistry of the actinoids is more complex than those of the lanthanoids'], 'Werner’s Theory of Coordination Compounds': ['what does the structure of a coordination compound consist of?', 'Who was the first to formulate his ideas about the structures of coordination compounds?', 'Why did he formulate his theory of coordination compounds?', 'what type of coordination is the theory of?', 'what was his name?'], 'General Properties of the Transition Elements': ['which chemical composition is the most common?', "what's the name of the periodic table?", 'the d–block occupies the large middle section of the periodic table PPPPPee', 'where is the d-block located?', 'what is the d-block??'], 'Integrated Rate Equations': ['how much pressure in the 

In [ ]:
print(reference_questions)

{'The Actinoids': ['Compare the chemistry of actinoids with that of the lanthanoids with specialreference to:  (i) electronic configuration (iii) oxidation state  (ii) atomic and ionic sizes and (iv) chemical reactivity.', 'How would you account for the following:  (i) Of the d4species, Cr2+is strongly reducing while manganese(III) is strongly oxidising.  (ii) Cobalt(II) is stable in aqueous solution but in the presence of complexing reagents it is easily oxidised.  (iii) The d1configuration is very unstable in ions.', "What is meant by 'disproportionation'? Give two examples of disproportionation reaction in aqueous  solution.", 'Which metal in the first series of transition metals exhibits +1 oxidationstate most frequently and why?', 'Calculate the number of unpaired electrons in the following gaseous ions: Mn3+, Cr3+, V3+ and Ti3+. Which one  of these is the most stable in aqueous solution?', 'What are inner transition elements? Decide which of the following atomic numbers are the a

In [ ]:
for query, questions in generated_questions.items():
  self_bleu = compute_self_bleu(questions, weights=(0.5, 0.5))  # BLEU-2
  print(f"Self-BLEU (BLEU-2): {self_bleu:.4f}")

  P, R, F1 = compute_bert_scores(questions, reference_questions[query])
  print(f"BERT-F1: {np.mean(F1):.4f}")

  rouge_scores = compute_rouge_scores(questions, reference_questions[query])
  print(f"ROUGE-L: {np.mean(rouge_scores):.4f}")

Self-BLEU (BLEU-2): 0.5843


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

BERT-F1: 0.8494
ROUGE-L: 0.1347
Self-BLEU (BLEU-2): 0.2262


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

BERT-F1: 0.8439
ROUGE-L: 0.1920
Self-BLEU (BLEU-2): 0.2681


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

### RAG + QG LLM (BART)

In [ ]:
!pip install transformers sentencepiece

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

bart_model = BartForConditionalGeneration.from_pretrained("voidful/bart-eqg-question-generator")
bart_tokenizer = BartTokenizer.from_pretrained("voidful/bart-eqg-question-generator")

def generate_bart_question(text):
    input_ids = bart_tokenizer.encode(text.strip(), return_tensors="pt", truncation=True, max_length=512)
    outputs = bart_model.generate(
        input_ids=bart_tokenizer.encode(text, return_tensors="pt"),
        do_sample=True,
        num_beams=5,
        top_k=50,
        top_p=0.95,
        num_return_sequences=5,
        temperature=0.7
    )
    return bart_tokenizer.decode(outputs[0], skip_special_tokens=True)

Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at voidful/bart-eqg-question-generator and are newly initialized: ['model.decoder.embed_tokens.weight', 'model.encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
generated_questions = {}
for sub_topic in test_df['Sub Topic'].unique():
  # --- Step 5: RAG Query ---
  embedder = SentenceTransformer("BAAI/bge-base-en-v1.5")
  query = sub_topic
  query_embedding = embedder.encode([query], normalize_embeddings=True)

  top_k = 5
  scores, indices = index.search(query_embedding, top_k)
  retrieved_chunks = [chunks[i] for i in indices[0]]

  # --- Step 6: Rerank Results ---
  reranker = CrossEncoder("BAAI/bge-reranker-large")
  pairs = [[query, passage] for passage in retrieved_chunks]
  rerank_scores = reranker.predict(pairs)

  ranked_passages = [x for _, x in sorted(zip(rerank_scores, retrieved_chunks), reverse=True)]
  context = "\n".join(ranked_passages)

  # --- Step 7: Use context to generate question ---

  generated_qs = generate_bart_question(context)
  print(generated_qs)
  #for i, q in enumerate(generated_qs, 1):
  #  print(f"{i}. {q}")

  generated_questions[sub_topic] = generated_qs

which which is is lives is is is belongs is ( is is to to tou ( which
which which is wants is is is belongs is is ( is ( ( which which is ( to
which which is is is belongs is is ( is ( ( which which is ( to tok
which which is is is has is is belongs is ( is ( ( to tok ( (
what which is is belongs is is is hass is ( ( which to to (ue to


In [ ]:
for query, questions in generated_questions.items():
  self_bleu = compute_self_bleu(questions, weights=(0.5, 0.5))  # BLEU-2
  print(f"Self-BLEU (BLEU-2): {self_bleu:.4f}")

  P, R, F1 = compute_bert_scores(questions, reference_questions[query])
  print(f"BERT-F1: {np.mean(F1):.4f}")

  rouge_scores = compute_rouge_scores(questions, reference_questions[query])
  print(f"ROUGE-L: {np.mean(rouge_scores):.4f}")

## Base Model with Fine Tuning

### Supervised Fine Tuning with Custom NCERT Data

In [ ]:
train_df.head()

,Sub Topic,Question,Answer
84,Importance and Applications of Coordination Co...,What is meant by the chelate effect? Give an e...,When a ligand attaches to the metal ion in a m...
95,Solubility,Calculate (a)molality (b)molarity and (c)mole ...,(a) Molar mass of KI = 39 + 127 = 166 g mol - ...
137,Abnormal Molar Masses,Henry's law constant for the molality of metha...,"Here, p = 760 mm Hg kH = 4.27 × 105 mm Hg ..."
211,Collision Theory of Chemical Reactions,The decomposition of hydrocarbon follows the e...,The given equation is k = (4.5 × 1011 s - 1) ...
86,Importance and Applications of Coordination Co...,How many ions are produced from the complex Co...,(iii) The given complex can be written as [Co(...


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd
import re

df = train_df.copy()

# Format input and output
df['input'] =  "generate question: " + df['Sub Topic']
df['target'] = df['Question']

# Convert to Dataset
dataset = Dataset.from_pandas(df[['input', 'target']]).train_test_split(test_size=0.1)

# Load tokenizer and model
model_name = "iarfmoose/t5-base-question-generator"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Tokenization function
def preprocess(examples):
    inputs = tokenizer(examples['input'], max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(examples['target'], max_length=64, truncation=True, padding="max_length")
    inputs['labels'] = targets['input_ids']
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=["input", "target"])

# Training configuration
training_args = TrainingArguments(
    output_dir="./vapour_pressure_qg_model",
    eval_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer
)

# Train and Save
trainer.train()
trainer.save_model("./qg_model")
tokenizer.save_pretrained("./qg_model")

Map:   0%|          | 0/154 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,2.567607
2,4.302500,2.346094
3,2.642300,2.282995
4,2.402300,2.250130


Epoch,Training Loss,Validation Loss
1,No log,2.567607
2,4.302500,2.346094
3,2.642300,2.282995
4,2.402300,2.250130


In [ ]:
generated_questions = {}
for sub_topic in test_df['Sub Topic'].unique():
  # --- Step 5: RAG Query ---
  query = sub_topic
  query_embedding = embedder.encode([query], normalize_embeddings=True)

  top_k = 5
  scores, indices = index.search(query_embedding, top_k)
  retrieved_chunks = [chunks[i] for i in indices[0]]

  # --- Step 6: Rerank Results ---
  reranker = CrossEncoder("BAAI/bge-reranker-large")
  pairs = [[query, passage] for passage in retrieved_chunks]
  rerank_scores = reranker.predict(pairs)

  ranked_passages = [x for _, x in sorted(zip(rerank_scores, retrieved_chunks), reverse=True)]
  context = "\n".join(ranked_passages)

  # --- Step 7: Use context to generate question ---
  model_name = "./qg_model"
  generated_qs = generate_t5_questions(model_name, context)
  for i, q in enumerate(generated_qs, 1):
    print(f"{i}. {q}")

  generated_questions[sub_topic] = generated_qs

1. What is the rate of reaction of a gas?
2. What is the temperature of the reaction?
3. What is the ratio of the energy of a reaction? 3.2.1 What is the ratio of the
4. How many times did the rate of reaction change to the mean temperature of the reaction? 3.3.3 How
5. When is the rate of a reaction quadrupled? 3.30 How many times does the
1. What is the rate of reaction in the case of a chemical reaction?
2. How does the speed of the reaction change between the reaction and the other? What is the atomic
3. What is the kinetic of the reaction?
4. The energy of the reaction is regulated by the kinetics of the reaction.
5. what is the ratio of the reaction rate of the molten gas in a molecule?
1. What is the difference between the DT b and the DT b?
2. what is the effect of a factor on the structure of a protein? What is the effect
3. What is the molar mass f? If the molar mass is a
4. What is the number of sandstones in a glass? What is the number of
5. What is the number of spheroids 

In [ ]:
for query, questions in generated_questions.items():
  self_bleu = compute_self_bleu(questions, weights=(0.5, 0.5))  # BLEU-2
  print(f"Self-BLEU (BLEU-2): {self_bleu:.4f}")

  P, R, F1 = compute_bert_scores(questions, reference_questions[query])
  print(f"BERT-F1: {np.mean(F1):.4f}")

  rouge_scores = compute_rouge_scores(questions, reference_questions[query])
  print(f"ROUGE-L: {np.mean(rouge_scores):.4f}")

Self-BLEU (BLEU-2): 0.5944


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

BERT-F1: 0.8593
ROUGE-L: 0.2516
Self-BLEU (BLEU-2): 0.4296


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

BERT-F1: 0.8669
ROUGE-L: 0.2599
Self-BLEU (BLEU-2): 0.3882


...

BERT-F1: 0.8625
ROUGE-L: 0.2383
Self-BLEU (BLEU-2): 0.3928


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

BERT-F1: 0.8458
ROUGE-L: 0.2001
Self-BLEU (BLEU-2): 0.4832


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

BERT-F1: 0.8595
ROUGE-L: 0.2131


In [ ]:
!cp -r ./qg_model /content/drive/MyDrive/Dataset/qg_model

### Fine Tuning using PEFT

In [ ]:
pip install peft accelerate transformers datasets

In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.1 MB/s eta 0:00:00


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
import bitsandbytes as bnb
print(bnb.__version__)


0.46.0


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from peft import get_peft_model, LoraConfig, TaskType

model_name = "iarfmoose/t5-base-question-generator"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],  # or other layers if needed
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, peft_config)

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType

# Assume train_df is already defined
df = train_df.copy()
df['input'] = "generate question: " + df['Sub Topic']
df['target'] = df['Question']

# Convert to Dataset
dataset = Dataset.from_pandas(df[['input', 'target']]).train_test_split(test_size=0.1)

# Tokenization function
def preprocess(examples):
    inputs = tokenizer(examples['input'], max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(examples['target'], max_length=64, truncation=True, padding="max_length")
    inputs['labels'] = targets['input_ids']
    return inputs

# Tokenize
tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=["input", "target"])

# Training configuration
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Dataset/qg_model_peft",
    eval_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
    fp16=True  # enable if using GPU with FP16
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer
)

# Train and Save
trainer.train()
model.save_pretrained("/content/drive/MyDrive/Dataset/qg_model_peft")
tokenizer.save_pretrained("/content/drive/MyDrive/Dataset/qg_model_peft")


Map:   0%|          | 0/154 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

/tmp/ipython-input-35-2297635730.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,9.611069
2,8.471500,7.272644
3,6.871500,5.738424
4,5.678800,4.808397


Epoch,Training Loss,Validation Loss
1,No log,9.611069
2,8.471500,7.272644
3,6.871500,5.738424
4,5.678800,4.808397


### Self Instruct Tuning

In [ ]:
import pandas as pd

df = train_df.copy()  # columns: topic, question
df["prompt"] = "Write a question about: " + df["Sub Topic"] + " Example: " + df["Question"]

In [ ]:
df["prompt"] = "Generate 5 questions on " + df["Sub Topic"] + "similar to " + df["Question"]
df["generated_qs"] = ""

In [ ]:
df.reset_index(drop=True, inplace=True)
df.head(10)

,Sub Topic,Question,Answer,prompt,generated_qs
0,Importance and Applications of Coordination Co...,What is meant by the chelate effect? Give an e...,When a ligand attaches to the metal ion in a m...,Generate 5 questions on Importance and Applica...,[Why do transition elements commonly display m...
1,Solubility,Calculate (a)molality (b)molarity and (c)mole ...,(a) Molar mass of KI = 39 + 127 = 166 g mol - ...,Generate 5 questions on Solubilitysimilar to C...,
2,Abnormal Molar Masses,Henry's law constant for the molality of metha...,"Here, p = 760 mm Hg kH = 4.27 × 105 mm Hg ...",Generate 5 questions on Abnormal Molar Massess...,
3,Collision Theory of Chemical Reactions,The decomposition of hydrocarbon follows the e...,The given equation is k = (4.5 × 1011 s - 1) ...,Generate 5 questions on Collision Theory of Ch...,
4,Importance and Applications of Coordination Co...,How many ions are produced from the complex Co...,(iii) The given complex can be written as [Co(...,Generate 5 questions on Importance and Applica...,
5,Galvanic Cells,Write the Nernst equation and emf of the follo...,"(i) For the given reaction, the Nernst equatio...",Generate 5 questions on Galvanic Cellssimilar ...,
6,Nernst Equation,If a current of 0.5 ampere flows through a met...,I = 0.5 A t = 2 hours = 2 × 60 × 60 s = 7200 ...,Generate 5 questions on Nernst Equationsimilar...,
7,The Actinoids,The chemistry of the actinoid elements is not ...,Lanthanoids primarily show three oxidation sta...,Generate 5 questions on The Actinoidssimilar t...,
8,The Lanthanoids,Describe the oxidising action of potassium dic...,acts as a very strong oxidising agent in the a...,Generate 5 questions on The Lanthanoidssimilar...,
9,Electrochemical Cell and Gibbs Energy of the R...,How much charge is required for the following ...,(i) Required charge = 3 F = 3 × 96487 C ...,Generate 5 questions on Electrochemical Cell a...,


In [ ]:
import openai
from openai import OpenAI
import re

API_KEY = "your_api_key"
openai.api_key = API_KEY

for i in range(len(df)):
  response = openai.chat.completions.create(
      model="gpt-4",
      messages=[
          {
              "role": "user",
              "content": [
                  {
                      "type": "text",
                      "text": df['prompt'][i]
                  }
              ]
          }
      ]
  )

  # Use regex to split into questions
  questions = re.split(r'\n?\d+\.\s+', response.choices[0].message.content)[1:]  # Skip the first empty match
  df['generated_qs'][i] = questions
  # Print or process the questions
  for i, q in enumerate(questions, 1):
      print(df['Sub Topic'][i], f"$ {q.strip()}")

/tmp/ipython-input-64-821369497.py:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['generated_qs'][i] = questions


Solubility $ What are coordination compounds and why are they important in chemical reactions?
Abnormal Molar Masses $ How do coordination compounds play a significant role in biological system? Provide an example.
Collision Theory of Chemical Reactions $ What are some of the significant industrial applications of coordination compounds?
Importance and Applications of Coordination Compounds $ What is a ligand in a coordination compound and how does it influence the properties of the compound?
Galvanic Cells $ Explain the concept of color in coordination compounds and how it can be used in qualitative analysis.


/tmp/ipython-input-64-821369497.py:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['generated_qs'][i] = questions


Solubility $ 2) Calculate the (a)molality (b)molarity and (c)mole fraction of CaCl2 in a solution if the density of the 25% (mass/mass) aqueous CaCl2 is 1.512 g mL-
Abnormal Molar Masses $ 3) Compute the (a)molality (b)molarity and (c)mole fraction of MgSO4 if the density of a 30% (mass/mass) aqueous MgSO4 solution is 1.301 g mL-
Collision Theory of Chemical Reactions $ 4) Ascertain the (a)molality (b)molarity and (c)mole fraction of Na2SO4 in a solution if the density of a 20% (mass/mass) aqueous Na2SO4 is 1.255 g mL-
Importance and Applications of Coordination Compounds $ 5) Find out the (a)molality (b)molarity and (c)mole fraction of K2CO3 if the density of a 18% (mass/mass) aqueous K2CO3 solution is 1.325 g mL-1.


/tmp/ipython-input-64-821369497.py:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['generated_qs'][i] = questions


...

/tmp/ipython-input-64-821369497.py:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['generated_qs'][i] = questions


Solubility $ How can you compute the mole fraction of sodium chloride in a solution containing 40% by mass in water?
Abnormal Molar Masses $ What is the method to find the mole fraction of ethanol in a solution containing 20% by mass in water?
Collision Theory of Chemical Reactions $ Could you explain the process of calculating the mole fraction of glucose in a solution containing 25% by mass in water?
Importance and Applications of Coordination Compounds $ Can you determine the mole fraction of sulfuric acid in a solution containing 50% by mass in water?
Galvanic Cells $ What is the procedure to calculate the mole fraction of acetic acid in a solution containing 35% by mass in water?


/tmp/ipython-input-64-821369497.py:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['generated_qs'][i] = questions


Solubility $ What is the rate constant of a reaction if the rate is 4 units, the concentration of reactant A is 2M, and of reactant B is 3M, given the rate law is R=k[A][B]?
Abnormal Molar Masses $ What is the overall order of a reaction if the rate law equation is expressed as R=k[A]^2[B]^1?
Collision Theory of Chemical Reactions $ Describe how the rate of reaction would change if the concentration of reactant A doubles in a reaction with a rate law R=k[A]^2[B].
Importance and Applications of Coordination Compounds $ For a reaction 2A + B -> Product, with a rate law given by R=k[A][B]^2, what is the effect on the rate of reaction if the concentration of B is halved?
Galvanic Cells $ If the rate law for a reaction is given by R=k[A][B]^2, how can you determine the order of the reaction with respect to each reactant?
Solubility $ What is the vapour pressure of a solution prepared by dissolving 2.0 g of a substance with a molar mass of 130,000 in 300 mL of water at 25°C?
Abnormal Molar M

/tmp/ipython-input-64-821369497.py:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['generated_qs'][i] = questions


In [ ]:
df.to_csv("/content/drive/MyDrive/Dataset/generated_questions.csv", index=False)

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Dataset/generated_questions.csv")

In [ ]:
!pip install bitsandbytes
from transformers import T5Tokenizer, T5ForConditionalGeneration, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
import bitsandbytes as bnb
print(bnb.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.7 MB/s eta 0:00:00


0.46.0


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from peft import get_peft_model, LoraConfig, TaskType

model_name = "iarfmoose/t5-base-question-generator"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],  # or other layers if needed
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, peft_config)

In [ ]:
import ast
df['generated_qs'] = df['generated_qs'].apply(ast.literal_eval)
df['generated_qs'][0]

['What are coordination compounds and why are they important in chemical reactions?',
 'How do coordination compounds play a significant role in biological system? Provide an example.',
 'What are some of the significant industrial applications of coordination compounds?',
 'What is a ligand in a coordination compound and how does it influence the properties of the compound?',
 'Explain the concept of color in coordination compounds and how it can be used in qualitative analysis.']

In [ ]:
df['questions_str'] = df['generated_qs'].apply(lambda x: '\n'.join(x))

In [ ]:
df['questions_str'][0]

'What are coordination compounds and why are they important in chemical reactions?\nHow do coordination compounds play a significant role in biological system? Provide an example.\nWhat are some of the significant industrial applications of coordination compounds?\nWhat is a ligand in a coordination compound and how does it influence the properties of the compound?\nExplain the concept of color in coordination compounds and how it can be used in qualitative analysis.'

In [ ]:
# Split multiline string into list of questions
df['generated_qs_list'] = df['questions_str'].apply(lambda x: x.strip().split('\n'))

# Explode list into separate rows
df_exploded = df.explode('generated_qs_list').rename(columns={'generated_qs_list': 'qs'})

# Optional: Reset index
df_exploded.reset_index(drop=True, inplace=True)

# View result
print(df_exploded)

                                             Sub Topic  \
0    Importance and Applications of Coordination Co...   
1    Importance and Applications of Coordination Co...   
2    Importance and Applications of Coordination Co...   
3    Importance and Applications of Coordination Co...   
4    Importance and Applications of Coordination Co...   
..                                                 ...   
936                Vapour Pressure of Liquid Solutions   
937                Vapour Pressure of Liquid Solutions   
938                Vapour Pressure of Liquid Solutions   
939                Vapour Pressure of Liquid Solutions   
940                Vapour Pressure of Liquid Solutions   

                                              Question  \
0    What is meant by the chelate effect? Give an e...   
1    What is meant by the chelate effect? Give an e...   
2    What is meant by the chelate effect? Give an e...   
3    What is meant by the chelate effect? Give an e...   
4    What is 

In [ ]:
df_exploded.head()

,Sub Topic,Question,Answer,prompt,generated_qs,questions_str,input,target,qs
0,Importance and Applications of Coordination Co...,What is meant by the chelate effect? Give an e...,When a ligand attaches to the metal ion in a m...,Generate 5 questions on Importance and Applica...,[What are coordination compounds and why are t...,What are coordination compounds and why are th...,Instruction: Generate a question based on the ...,What is meant by the chelate effect? Give an e...,What are coordination compounds and why are th...
1,Importance and Applications of Coordination Co...,What is meant by the chelate effect? Give an e...,When a ligand attaches to the metal ion in a m...,Generate 5 questions on Importance and Applica...,[What are coordination compounds and why are t...,What are coordination compounds and why are th...,Instruction: Generate a question based on the ...,What is meant by the chelate effect? Give an e...,How do coordination compounds play a significa...
2,Importance and Applications of Coordination Co...,What is meant by the chelate effect? Give an e...,When a ligand attaches to the metal ion in a m...,Generate 5 questions on Importance and Applica...,[What are coordination compounds and why are t...,What are coordination compounds and why are th...,Instruction: Generate a question based on the ...,What is meant by the chelate effect? Give an e...,What are some of the significant industrial ap...
3,Importance and Applications of Coordination Co...,What is meant by the chelate effect? Give an e...,When a ligand attaches to the metal ion in a m...,Generate 5 questions on Importance and Applica...,[What are coordination compounds and why are t...,What are coordination compounds and why are th...,Instruction: Generate a question based on the ...,What is meant by the chelate effect? Give an e...,What is a ligand in a coordination compound an...
4,Importance and Applications of Coordination Co...,What is meant by the chelate effect? Give an e...,When a ligand attaches to the metal ion in a m...,Generate 5 questions on Importance and Applica...,[What are coordination compounds and why are t...,What are coordination compounds and why are th...,Instruction: Generate a question based on the ...,What is meant by the chelate effect? Give an e...,Explain the concept of color in coordination c...


In [ ]:
df_exploded.to_csv("/content/drive/MyDrive/Dataset/generated_exploded_questions.csv", index=False)

In [ ]:
generated_questions = {}
df_exploded['Context'] = ""
for sub_topic in df_exploded['Sub Topic'].unique():
  # --- Step 5: RAG Query ---
  query = sub_topic
  query_embedding = embedder.encode([query], normalize_embeddings=True)

  top_k = 5
  scores, indices = index.search(query_embedding, top_k)
  retrieved_chunks = [chunks[i] for i in indices[0]]

  # --- Step 6: Rerank Results ---
  reranker = CrossEncoder("BAAI/bge-reranker-large")
  pairs = [[query, passage] for passage in retrieved_chunks]
  rerank_scores = reranker.predict(pairs)

  ranked_passages = [x for _, x in sorted(zip(rerank_scores, retrieved_chunks), reverse=True)]
  context = "\n".join(ranked_passages)

  df_exploded.loc[df_exploded['Sub Topic'] == sub_topic, 'Context'] = context


config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/34.0k [00:00<?, ?B/s]

In [ ]:
df_exploded['input'] = (
    "Instruction: Generate a question based on the topic and examples.\n"
    "Topic: " + df_exploded['Sub Topic'] +"\n"+
    "Examples:\n" +
    df_exploded['qs']+"\n"+
    "Context" + df_exploded['Context']
)

df_exploded['target'] = df_exploded['Question']

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
from transformers import EarlyStoppingCallback
"""
# Assume train_df is already defined
df = train_df.copy()
df['input'] = "generate question: " + df['Sub Topic']
df['target'] = df['Question']
"""
# Convert to Dataset
dataset = Dataset.from_pandas(df_exploded[['input', 'target']]).train_test_split(test_size=0.1)
model_name = "iarfmoose/t5-base-question-generator"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Tokenization function
def preprocess(examples):
    inputs = tokenizer(examples['input'], max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(examples['target'], max_length=64, truncation=True, padding="max_length")
    inputs['labels'] = targets['input_ids']
    return inputs

# Tokenize
tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=["input", "target"])

# Training configuration
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Dataset/qg_model_instruction_tuned_peft",
    eval_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=20,
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
    fp16=True  # enable if using GPU with FP16
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Train and Save
trainer.train()
model.save_pretrained("/content/drive/MyDrive/Dataset/qg_model_instruction_tuned_peft")
tokenizer.save_pretrained("/content/drive/MyDrive/Dataset/qg_model_instruction_tuned_peft")


Map:   0%|          | 0/846 [00:00<?, ? examples/s]

Map:   0%|          | 0/95 [00:00<?, ? examples/s]

/tmp/ipython-input-44-704841009.py:44: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,1.522600,1.322266
2,1.125400,0.991622
3,0.911100,0.803654


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

checkpoint = '/content/drive/MyDrive/Dataset/qg_model_instruction_tuned_peft/checkpoint-636/'
model = T5ForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = T5Tokenizer.from_pretrained(checkpoint)

def generate_t5_questions(query, context="", num_questions = 5) -> list:
    #tokenizer = T5Tokenizer.from_pretrained(model_name)
    #model = T5ForConditionalGeneration.from_pretrained(model_name)

    input_text = "generate question on: " + query #+ "context:" + context.strip()
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)

    outputs = model.generate(
        input_ids=input_ids,
        #max_length=64,
        num_return_sequences=num_questions,
        do_sample=True,
        top_k=40,
        top_p=0.9,
        temperature=0.8,
        #early_stopping=True
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

In [ ]:
def generate_questions_for_test_df(test_df):
  generated_questions = {}
  for sub_topic in test_df['Sub Topic'].unique():
    # --- Step 5: RAG Query ---
    query = sub_topic
    query_embedding = embedder.encode([query], normalize_embeddings=True)

    top_k = 5
    scores, indices = index.search(query_embedding, top_k)
    retrieved_chunks = [chunks[i] for i in indices[0]]

    # --- Step 6: Rerank Results ---
    reranker = CrossEncoder("BAAI/bge-reranker-large")
    pairs = [[query, passage] for passage in retrieved_chunks]
    rerank_scores = reranker.predict(pairs)

    ranked_passages = [x for _, x in sorted(zip(rerank_scores, retrieved_chunks), reverse=True)]
    context = "\n".join(ranked_passages)

    # --- Step 7: Use context to generate question ---
    generated_qs = generate_t5_questions(query, context)
    for i, q in enumerate(generated_qs, 1):
      print(f"{i}. {q}")

    generated_questions[sub_topic] = generated_qs
  return generated_questions

In [ ]:
generated_qs = generate_questions_for_test_df(test_df)

1. The rate of a reaction is dependent on the temperature of the ice.
2. Temperature Dependence of the Rate of a Reaction :
3. The rate of a reaction depends on the following factors: Temperature, Time, and Temperature
4. The rate of a reaction depends on the following factors: Temperature Dependence of the Rate of
5. The following data are obtained from the temperature dependence of the rate of a reaction:
1. What is the difference between a cis- reaction and a cis- reaction
2. The theory of collisions and reaction in the presence of carbon is based on the following:
3. Which of the following will be the main thrust of the debate on: Collision Theory of Chemical
4. What is the theory of collisions?
5. The Theory of Collision of Chemical Reactions
1. Why are there so many abnormal masses of molar mass?
2. What are abnormal molar masses?
3. What are abnormal Molar Masses?
4. How many molar masses do you know of that are caused by astro-
5. A number of abnormal Molar Masses have been ide

In [ ]:
for query, questions in generated_qs.items():
  self_bleu = compute_self_bleu(questions, weights=(0.5, 0.5))  # BLEU-2
  print(f"Self-BLEU (BLEU-2): {self_bleu:.4f}")

  P, R, F1 = compute_bert_scores(questions, reference_questions[query])
  print(f"BERT-F1: {np.mean(F1):.4f}")

  rouge_scores = compute_rouge_scores(questions, reference_questions[query])
  print(f"ROUGE-L: {np.mean(rouge_scores):.4f}")

Self-BLEU (BLEU-2): 0.6245


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

BERT-F1: 0.8365
ROUGE-L: 0.1863
Self-BLEU (BLEU-2): 0.2458


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

BERT-F1: 0.8223
ROUGE-L: 0.0744
Self-BLEU (BLEU-2): 0.3338


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

BERT-F1: 0.8486
ROUGE-L: 0.2079


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 201 to 93
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sub Topic  44 non-null     object
 1   Question   44 non-null     object
 2   Answer     44 non-null     object
dtypes: object(3)
memory usage: 1.4+ KB


### Self Instruction Tuning combined with PEFT

In [ ]:
import pandas as pd

df = train_df.copy()  # columns: topic, question
df["prompt"] = "Write a question about: " + df["Sub Topic"] + " Example: " + df["Question"]

In [ ]:
df["prompt"] = "Generate 5 questions on " + df["Sub Topic"] + "similar to " + df["Question"]
df["generated_qs"] = ""

In [ ]:
df.reset_index(drop=True, inplace=True)
df.head(10)

,Sub Topic,Question,Answer,prompt,generated_qs
0,Importance and Applications of Coordination Co...,What is meant by the chelate effect? Give an e...,When a ligand attaches to the metal ion in a m...,Generate 5 questions on Importance and Applica...,[Why do transition elements commonly display m...
1,Solubility,Calculate (a)molality (b)molarity and (c)mole ...,(a) Molar mass of KI = 39 + 127 = 166 g mol - ...,Generate 5 questions on Solubilitysimilar to C...,
2,Abnormal Molar Masses,Henry's law constant for the molality of metha...,"Here, p = 760 mm Hg kH = 4.27 × 105 mm Hg ...",Generate 5 questions on Abnormal Molar Massess...,
3,Collision Theory of Chemical Reactions,The decomposition of hydrocarbon follows the e...,The given equation is k = (4.5 × 1011 s - 1) ...,Generate 5 questions on Collision Theory of Ch...,
4,Importance and Applications of Coordination Co...,How many ions are produced from the complex Co...,(iii) The given complex can be written as [Co(...,Generate 5 questions on Importance and Applica...,
5,Galvanic Cells,Write the Nernst equation and emf of the follo...,"(i) For the given reaction, the Nernst equatio...",Generate 5 questions on Galvanic Cellssimilar ...,
6,Nernst Equation,If a current of 0.5 ampere flows through a met...,I = 0.5 A t = 2 hours = 2 × 60 × 60 s = 7200 ...,Generate 5 questions on Nernst Equationsimilar...,
7,The Actinoids,The chemistry of the actinoid elements is not ...,Lanthanoids primarily show three oxidation sta...,Generate 5 questions on The Actinoidssimilar t...,
8,The Lanthanoids,Describe the oxidising action of potassium dic...,acts as a very strong oxidising agent in the a...,Generate 5 questions on The Lanthanoidssimilar...,
9,Electrochemical Cell and Gibbs Energy of the R...,How much charge is required for the following ...,(i) Required charge = 3 F = 3 × 96487 C ...,Generate 5 questions on Electrochemical Cell a...,


In [ ]:
import openai
from openai import OpenAI
import re

API_KEY = "your_api_key"
openai.api_key = API_KEY

for i in range(len(df)):
  response = openai.chat.completions.create(
      model="gpt-4",
      messages=[
          {
              "role": "user",
              "content": [
                  {
                      "type": "text",
                      "text": df['prompt'][i]
                  }
              ]
          }
      ]
  )

  # Use regex to split into questions
  questions = re.split(r'\n?\d+\.\s+', response.choices[0].message.content)[1:]  # Skip the first empty match
  df['generated_qs'][i] = questions
  # Print or process the questions
  for i, q in enumerate(questions, 1):
      print(df['Sub Topic'][i], f"$ {q.strip()}")

/tmp/ipython-input-64-821369497.py:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['generated_qs'][i] = questions


Solubility $ What are coordination compounds and why are they important in chemical reactions?
Abnormal Molar Masses $ How do coordination compounds play a significant role in biological system? Provide an example.
Collision Theory of Chemical Reactions $ What are some of the significant industrial applications of coordination compounds?
Importance and Applications of Coordination Compounds $ What is a ligand in a coordination compound and how does it influence the properties of the compound?
Galvanic Cells $ Explain the concept of color in coordination compounds and how it can be used in qualitative analysis.


/tmp/ipython-input-64-821369497.py:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['generated_qs'][i] = questions


Solubility $ 2) Calculate the (a)molality (b)molarity and (c)mole fraction of CaCl2 in a solution if the density of the 25% (mass/mass) aqueous CaCl2 is 1.512 g mL-
Abnormal Molar Masses $ 3) Compute the (a)molality (b)molarity and (c)mole fraction of MgSO4 if the density of a 30% (mass/mass) aqueous MgSO4 solution is 1.301 g mL-
Collision Theory of Chemical Reactions $ 4) Ascertain the (a)molality (b)molarity and (c)mole fraction of Na2SO4 in a solution if the density of a 20% (mass/mass) aqueous Na2SO4 is 1.255 g mL-
Importance and Applications of Coordination Compounds $ 5) Find out the (a)molality (b)molarity and (c)mole fraction of K2CO3 if the density of a 18% (mass/mass) aqueous K2CO3 solution is 1.325 g mL-1.


/tmp/ipython-input-64-821369497.py:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['generated_qs'][i] = questions


...

/tmp/ipython-input-64-821369497.py:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['generated_qs'][i] = questions


Solubility $ How can you compute the mole fraction of sodium chloride in a solution containing 40% by mass in water?
Abnormal Molar Masses $ What is the method to find the mole fraction of ethanol in a solution containing 20% by mass in water?
Collision Theory of Chemical Reactions $ Could you explain the process of calculating the mole fraction of glucose in a solution containing 25% by mass in water?
Importance and Applications of Coordination Compounds $ Can you determine the mole fraction of sulfuric acid in a solution containing 50% by mass in water?
Galvanic Cells $ What is the procedure to calculate the mole fraction of acetic acid in a solution containing 35% by mass in water?


/tmp/ipython-input-64-821369497.py:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['generated_qs'][i] = questions


Solubility $ What is the rate constant of a reaction if the rate is 4 units, the concentration of reactant A is 2M, and of reactant B is 3M, given the rate law is R=k[A][B]?
Abnormal Molar Masses $ What is the overall order of a reaction if the rate law equation is expressed as R=k[A]^2[B]^1?
Collision Theory of Chemical Reactions $ Describe how the rate of reaction would change if the concentration of reactant A doubles in a reaction with a rate law R=k[A]^2[B].
Importance and Applications of Coordination Compounds $ For a reaction 2A + B -> Product, with a rate law given by R=k[A][B]^2, what is the effect on the rate of reaction if the concentration of B is halved?
Galvanic Cells $ If the rate law for a reaction is given by R=k[A][B]^2, how can you determine the order of the reaction with respect to each reactant?
Solubility $ What is the vapour pressure of a solution prepared by dissolving 2.0 g of a substance with a molar mass of 130,000 in 300 mL of water at 25°C?
Abnormal Molar M

/tmp/ipython-input-64-821369497.py:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['generated_qs'][i] = questions


In [ ]:
df.to_csv("/content/drive/MyDrive/Dataset/generated_questions.csv", index=False)

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Dataset/generated_questions.csv")

In [ ]:
!pip install bitsandbytes
from transformers import T5Tokenizer, T5ForConditionalGeneration, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
import bitsandbytes as bnb
print(bnb.__version__)

0.46.0


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from peft import get_peft_model, LoraConfig, TaskType

model_name = "iarfmoose/t5-base-question-generator"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],  # or other layers if needed
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, peft_config)

In [ ]:
import pandas as pd
df_exploded = pd.read_csv("/content/drive/MyDrive/Dataset/generated_exploded_questions.csv")

In [ ]:
df_exploded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941 entries, 0 to 940
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Sub Topic      941 non-null    object
 1   Question       941 non-null    object
 2   Answer         941 non-null    object
 3   prompt         941 non-null    object
 4   generated_qs   941 non-null    object
 5   questions_str  929 non-null    object
 6   input          941 non-null    object
 7   target         941 non-null    object
 8   qs             833 non-null    object
 9   Context        941 non-null    object
dtypes: object(10)
memory usage: 73.6+ KB


In [ ]:
df['questions_str'] = df['generated_qs'].apply(lambda x: '\n'.join(x))

In [ ]:
df['questions_str'][0]

'What are coordination compounds and why are they important in chemical reactions?\nHow do coordination compounds play a significant role in biological system? Provide an example.\nWhat are some of the significant industrial applications of coordination compounds?\nWhat is a ligand in a coordination compound and how does it influence the properties of the compound?\nExplain the concept of color in coordination compounds and how it can be used in qualitative analysis.'

In [ ]:
# Split multiline string into list of questions
df['generated_qs_list'] = df['questions_str'].apply(lambda x: x.strip().split('\n'))

# Explode list into separate rows
df_exploded = df.explode('generated_qs_list').rename(columns={'generated_qs_list': 'qs'})

# Optional: Reset index
df_exploded.reset_index(drop=True, inplace=True)

# View result
print(df_exploded)

                                             Sub Topic  \
0    Importance and Applications of Coordination Co...   
1    Importance and Applications of Coordination Co...   
2    Importance and Applications of Coordination Co...   
3    Importance and Applications of Coordination Co...   
4    Importance and Applications of Coordination Co...   
..                                                 ...   
936                Vapour Pressure of Liquid Solutions   
937                Vapour Pressure of Liquid Solutions   
938                Vapour Pressure of Liquid Solutions   
939                Vapour Pressure of Liquid Solutions   
940                Vapour Pressure of Liquid Solutions   

                                              Question  \
0    What is meant by the chelate effect? Give an e...   
1    What is meant by the chelate effect? Give an e...   
2    What is meant by the chelate effect? Give an e...   
3    What is meant by the chelate effect? Give an e...   
4    What is 

In [ ]:
df_exploded.to_csv("/content/drive/MyDrive/Dataset/generated_exploded_questions.csv", index=False)

In [ ]:
generated_questions = {}
df_exploded['Context'] = ""
for sub_topic in df_exploded['Sub Topic'].unique():
  # --- Step 5: RAG Query ---
  query = sub_topic
  query_embedding = embedder.encode([query], normalize_embeddings=True)

  top_k = 5
  scores, indices = index.search(query_embedding, top_k)
  retrieved_chunks = [chunks[i] for i in indices[0]]

  # --- Step 6: Rerank Results ---
  reranker = CrossEncoder("BAAI/bge-reranker-large")
  pairs = [[query, passage] for passage in retrieved_chunks]
  rerank_scores = reranker.predict(pairs)

  ranked_passages = [x for _, x in sorted(zip(rerank_scores, retrieved_chunks), reverse=True)]
  context = "\n".join(ranked_passages)

  df_exploded.loc[df_exploded['Sub Topic'] == sub_topic, 'Context'] = context


config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/34.0k [00:00<?, ?B/s]

In [ ]:
df_exploded['input'] = (
    "Instruction: Generate a question based on the topic and examples.\n"
    "Topic: " + df_exploded['Sub Topic'] +"\n"+
    "Examples:\n" +
    df_exploded['qs']+"\n"+
    "Context" + df_exploded['Context']
)

df_exploded['target'] = df_exploded['Question']

In [ ]:
type(df_exploded['input'][0])

str

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
from transformers import EarlyStoppingCallback
"""
# Assume train_df is already defined
df = train_df.copy()
df['input'] = "generate question: " + df['Sub Topic']
df['target'] = df['Question']
"""
# Convert to Dataset
dataset = Dataset.from_pandas(df_exploded[['input', 'target']]).train_test_split(test_size=0.1)

# Tokenization function
def preprocess(examples):
    print(type(examples['input']))
    inputs = tokenizer(examples['input'], max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(examples['target'], max_length=64, truncation=True, padding="max_length")
    inputs['labels'] = targets['input_ids']
    return inputs

# Tokenize
tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=["input", "target"])

# Training configuration
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Dataset/qg_model_instruction_tuned_peft",
    eval_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=20,
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
    fp16=True  # enable if using GPU with FP16
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Train and Save
trainer.train()
model.save_pretrained("/content/drive/MyDrive/Dataset/qg_model_instruction_tuned_peft")
tokenizer.save_pretrained("/content/drive/MyDrive/Dataset/qg_model_instruction_tuned_peft")


Map:   0%|          | 0/846 [00:00<?, ? examples/s]

<class 'list'>


Map:   0%|          | 0/95 [00:00<?, ? examples/s]

<class 'list'>


/tmp/ipython-input-58-1899143480.py:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,2.585700,2.036716
2,2.153900,1.814778
3,1.987400,1.730253
4,2.017700,1.672679
5,1.791500,1.627630
6,1.830600,1.584432
7,1.792600,1.551706


In [ ]:
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel, PeftConfig

# Path to the PEFT (LoRA) checkpoint directory
peft_checkpoint = '/content/drive/MyDrive/Dataset/qg_model_instruction_tuned_peft/checkpoint-1484/'

# Step 1: Load the PEFT config
peft_config = PeftConfig.from_pretrained(peft_checkpoint)

# Step 2: Load the base model (same one used during PEFT fine-tuning)
base_model = AutoModelForSeq2SeqLM.from_pretrained(peft_config.base_model_name_or_path)

# Step 3: Load the PEFT adapter weights into the base model
model = PeftModel.from_pretrained(base_model, peft_checkpoint)
model.eval()

# Step 4: Load tokenizer (usually same as base model)
tokenizer = T5Tokenizer.from_pretrained(peft_config.base_model_name_or_path)

# Question generation function
def generate_t5_questions(query, context="", num_questions=5) -> list:
    input_text = "generate question on: " + query
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)

    outputs = model.generate(
        input_ids=input_ids,
        num_return_sequences=num_questions,
        do_sample=True,
        top_k=40,
        top_p=0.9,
        temperature=0.8,
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
def generate_questions_for_test_df(test_df):
  generated_questions = {}
  for sub_topic in test_df['Sub Topic'].unique():
    # --- Step 5: RAG Query ---
    query = sub_topic
    query_embedding = embedder.encode([query], normalize_embeddings=True)

    top_k = 5
    scores, indices = index.search(query_embedding, top_k)
    retrieved_chunks = [chunks[i] for i in indices[0]]

    # --- Step 6: Rerank Results ---
    reranker = CrossEncoder("BAAI/bge-reranker-large")
    pairs = [[query, passage] for passage in retrieved_chunks]
    rerank_scores = reranker.predict(pairs)

    ranked_passages = [x for _, x in sorted(zip(rerank_scores, retrieved_chunks), reverse=True)]
    context = "\n".join(ranked_passages)

    # --- Step 7: Use context to generate question ---
    generated_qs = generate_t5_questions(query, context)
    for i, q in enumerate(generated_qs, 1):
      print(f"{i}. {q}")

    generated_questions[sub_topic] = generated_qs
  return generated_questions

generated_qs = generate_questions_for_test_df(test_df)

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/34.0k [00:00<?, ?B/s]

1. In this respect, the following is expected: - - Temperature Dependence of the Rate
2. When the rate of a reaction is high, it is likely that the rate of a reaction
3. On: The Temporal Dependence of the Rate of a Reaction.
4. The rate of a reaction is dependent on the following: Temperature Dependence of the Rate of
5. The rate of a reaction depends on the following parameters:
1. Reliability of acetonitrile ions :
2. The following results are derived from the following physics : Collision Theory of Chemical Re
3. What is the reaction theory of a chemical reaction?
4. Which of the following is the most important factor?
5. What is the correlation between a metal's temperature and its mass?
1. The following is a diagram of the chemical compositions of the following: Extending Concentration
2. The following equations will be used to describe the following: Concentration of Solution in Solution and Concentr
3. What is the effect of the following on the concentration of solutions of the 

In [ ]:
for query, questions in generated_qs.items():
  self_bleu = compute_self_bleu(questions, weights=(0.5, 0.5))  # BLEU-2
  print(f"Self-BLEU (BLEU-2): {self_bleu:.4f}")

  P, R, F1 = compute_bert_scores(questions, reference_questions[query])
  print(f"BERT-F1: {np.mean(F1):.4f}")

  rouge_scores = compute_rouge_scores(questions, reference_questions[query])
  print(f"ROUGE-L: {np.mean(rouge_scores):.4f}")

Self-BLEU (BLEU-2): 0.5551


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

BERT-F1: 0.8286
ROUGE-L: 0.1400
Self-BLEU (BLEU-2): 0.0929


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

BERT-F1: 0.8158


ValueError: Hypothesis is empty.